#### Логистическая регрессия (TFIDF+SVD)

In [2]:
import numpy as np
import pandas as pd

In [3]:
from utils import *

In [4]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
df = pd.read_feather('data/prep.ftr')
X, y = df.drop(columns='topic'), df['topic']
params = {'stratify': y, 'test_size': 0.2, 'random_state': 42}
X_train, X_valid, y_train, y_valid = train_test_split(X, y, **params)

* *загружаем предобработанный датафрейм*
* *разделяем на 2е части: для обучения и валидации*
* *80% данных в обучении, стратификация по классам*

In [5]:
logreg_params = {
    'solver': 'liblinear'
}

svd_params = {
    'n_iter': 5, 
    'random_state': 42, 
    'algorithm': 'randomized'
}

tfidf_params = {
    'max_df': 0.8, 
    'min_df': 0.0004, 
    'dtype': np.float32, 
    'ngram_range': (1,2), 
    'token_pattern': None, 
    'tokenizer': str.split
}

sca = ('sca', StandardScaler())
vec = ('vec', TfidfVectorizer(**tfidf_params))
clf = ('clf', LogisticRegression(**logreg_params))

text_svd = ('text_svd', TruncatedSVD(50, **svd_params))
title_svd = ('title_svd', TruncatedSVD(300, **svd_params))

text_vec = ('text_vec', Pipeline([vec, text_svd]), 'text')
title_vec = ('title_vec', Pipeline([vec, title_svd]), 'title')
trans = ('trans', ColumnTransformer([text_vec, title_vec], remainder='passthrough'))

logreg = Pipeline([trans, sca, clf])

In [6]:
TfidfVectorizer(**tfidf_params).fit_transform(X_train['text']).shape

(345726, 35333)

* *собираем pipeline: TFIDF -> SVD -> Scaler -> LogReg*
* *независимая векторизация title и text (50 и 300 фичей соответственно)*
* *TFIDF: униграммы и биграммы, после подбора min_df и max_df получаем 35333 токенов в словаре*
* *на входе также будут title_len и text_len из предобработанного датафрейма (тоже попадут в Scaler)*

In [ ]:
%time logreg.fit(X_train, y_train)

* *обучение модели на 80% данных занимает ~15мин*

In [16]:
%time y_pred = logreg.predict(X_valid)

CPU times: user 28.3 s, sys: 456 ms, total: 28.8 s
Wall time: 28.8 s


In [9]:
accuracy_score(y_valid, y_pred)

0.9043409848204369

In [10]:
f1_score(y_valid, y_pred, average='macro')

0.8939069567118967

In [11]:
print(confusion_matrix(y_valid, y_pred))

[[ 6698   377   805   447   101   505]
 [  288  9708   522   103    33   105]
 [  387   298 25461   531   108   539]
 [  384   160   641  9130    23   289]
 [   58    27   160    30 12569    39]
 [  330    74   662   177    65 14598]]


In [12]:
print(classification_report(y_valid, y_pred))

              precision    recall  f1-score   support

           0       0.82      0.75      0.78      8933
           1       0.91      0.90      0.91     10759
           2       0.90      0.93      0.92     27324
           3       0.88      0.86      0.87     10627
           4       0.97      0.98      0.98     12883
           5       0.91      0.92      0.91     15906

    accuracy                           0.90     86432
   macro avg       0.90      0.89      0.89     86432
weighted avg       0.90      0.90      0.90     86432



In [6]:
list(load_json('data/labels.json').keys())

['Интернет и СМИ', 'Культура', 'Мир', 'Наука и техника', 'Спорт', 'Экономика']

* *наименее точно модель классифицирует класс 0 (Интернет и СМИ)*
* *данный класс имеет наименьшее количество объектов в обучающей выборке (и в датасете)*

In [15]:
dump_pickle(logreg, 'data/logreg.bin')

* *данная модель занимает достаточно много места на диске и в памяти (файл модели ~400Мб)*
* *в данном случае внутри pipeline имеем обученный TFIDF и SVD, отсюда и такой размер модели*
* *кроме того, TFIDF и SVD внутри модели скорее всего приведут к более низкой производительности*